# Build Validation Table for NPMRDS
For each of the 9 SACSIM time periods, provide:
* Average speed
* Number of epochs
* Standard error of the speed (standard deviation / mean)

In [40]:
import os
from time import perf_counter as perf

import pandas as pd
import urllib
import sqlalchemy as sqla # needed to run pandas df.to_sql() function
    
# extract SQL Server query results into a pandas dataframe   
def sqlqry_to_df(query_str, dbname, servername='SQL-SVR', trustedconn='yes'):     

    conn_str = "DRIVER={ODBC Driver 17 for SQL Server};" \
        f"SERVER={servername};" \
        f"DATABASE={dbname};" \
        f"Trusted_Connection={trustedconn}"
        
    conn_str = urllib.parse.quote_plus(conn_str)
    engine = sqla.create_engine(f"mssql+pyodbc:///?odbc_connect={conn_str}")
       
    start_time = perf()

    # create SQL table from the dataframe
    print("Executing query. Results loading into dataframe...")
    df = pd.read_sql_query(sql=query_str, con=engine)
    rowcnt = df.shape[0]
    
    et_mins = round((perf() - start_time) / 60, 2)
    print(f"Successfully executed query in {et_mins} minutes. {rowcnt} rows loaded into dataframe.")
    
    return df

def get_prd_df(qry_file, db_name, speed_tbl, tmc_tbl, prd_tag, prd_hrs):
    
    with open(qry_file, 'r') as f:
        q_str = f.read()
        
    q_str_formatted = q_str.format(speed_tbl, prd_hrs, prd_tag, tmc_tbl)
    
    prd_df = sqlqry_to_df(q_str_formatted, db_name)
    
    return prd_df
    

In [41]:
query_file = 'cong_speed_model_prds.sql'
# query_file = 'cong_speed_model_prds_err_rngs.sql'

db = 'NPMRDS'
tbl_speeds = 'npmrds_2016_alltmc_paxtruck_comb'
tbl_tmcs = 'npmrds_2020_alltmc_txt'
join_key = 'tmc'

prds = {'h07':[7], 'h08':[8], 'h09':[9], 'md5':[10, 11, 12, 13, 14], 
        'h15':[15], 'h16':[16], 'h17':[17], 'ev2':[18, 19], 
        'n11': [20, 21, 22, 23, 0, 1, 2, 3, 4, 5, 6]}

df_master = pd.DataFrame()

# for i, prdtag in enumerate(['h07']): # for testing
for i, prdtag in enumerate(prds.keys()):
    print(f"loading in data for {prdtag}...")
    prd_hours = ','.join(str(i) for i in prds[prdtag])
    if i == 0:
        df_master = get_prd_df(qry_file=query_file, db_name=db,
                               speed_tbl=tbl_speeds, tmc_tbl=tbl_tmcs, 
                               prd_tag=prdtag, prd_hrs=prd_hours)
    else:
        df_p = get_prd_df(qry_file=query_file, db_name=db,
                               speed_tbl=tbl_speeds, tmc_tbl=tbl_tmcs, 
                               prd_tag=prdtag, prd_hrs=prd_hours)
        
        df_master = df_master.merge(df_p, on=join_key)
        
        
print("successfully created dataframe for speeds in all sacsim time periods.")
        

    


loading in data for h07...
Executing query. Results loading into dataframe...
Successfully executed query in 0.26 minutes. 7495 rows loaded into dataframe.
loading in data for h08...
Executing query. Results loading into dataframe...
Successfully executed query in 0.23 minutes. 7495 rows loaded into dataframe.
loading in data for h09...
Executing query. Results loading into dataframe...
Successfully executed query in 0.22 minutes. 7495 rows loaded into dataframe.
loading in data for md5...
Executing query. Results loading into dataframe...
Successfully executed query in 0.36 minutes. 7495 rows loaded into dataframe.
loading in data for h15...
Executing query. Results loading into dataframe...
Successfully executed query in 0.22 minutes. 7495 rows loaded into dataframe.
loading in data for h16...
Executing query. Results loading into dataframe...
Successfully executed query in 0.23 minutes. 7495 rows loaded into dataframe.
loading in data for h17...
Executing query. Results loading into

In [42]:
df_master.head()
# cols = [f"spd_stderr_{prd}" for prd in list(prds.keys())]

# df_master[cols].mean()

,tmc,avspd_h07,epcnt_h07,stdev_h07,spd_stderr_h07,avspd_h08,epcnt_h08,stdev_h08,spd_stderr_h08,avspd_h09,...,stdev_h17,spd_stderr_h17,avspd_ev2,epcnt_ev2,stdev_ev2,spd_stderr_ev2,avspd_n11,epcnt_n11,stdev_n11,spd_stderr_n11
0,105+16772,36.214347,179,8.286380,0.228815,34.133528,175,9.402569,0.275464,35.061863,...,7.703417,0.220103,36.546295,243,8.465070,0.231626,40.739203,548,8.375490,0.205588
1,105+16190,32.706430,49,8.507402,0.260114,37.189405,98,5.666340,0.152364,34.235105,...,5.989804,0.151397,38.757190,91,6.486252,0.167356,40.609054,139,6.137053,0.151125
2,105+06726,55.726842,203,3.093015,0.055503,55.288400,204,2.715014,0.049106,50.842853,...,3.220826,0.059057,55.082381,401,3.698512,0.067145,55.435199,1570,4.271279,0.077050
3,105-08101,43.612691,75,11.354380,0.260346,49.401661,82,8.879582,0.179743,48.701124,...,6.982993,0.129089,54.651833,139,6.320305,0.115647,42.928979,286,10.515034,0.244940
4,105+06827,41.076997,196,5.468751,0.133134,42.660455,190,5.171167,0.121217,44.074143,...,6.078383,0.139741,43.809808,207,7.330547,0.167327,46.373650,657,5.686750,0.122629


In [43]:
# join to link feature class and export to GIS feature class
from arcgis.features import GeoAccessor, GeoSeriesAccessor
import datetime as dt

input_link_fc = r'Q:\SACSIM23\Network\SM23GIS\MN_link_forConflation_YZ\ConflationResults.gdb\conflation_INRIX_20211207'
links_jnky = 'Tmc'
link_fc_cols = ['OBJECTID', 'A', 'B', 'A_B', 'CAPC20', 'NAME', links_jnky,
       'RoadName', 'FwyTag', 'RoadNumber', 'Type', 'Mis_cf_check', 'SHAPE']


output_fgdb = r'Q:\SACSIM23\Network\SM23GIS\SM23Testing.gdb'
speed_data_year = 2016
TMC_yr = 2020

#-----------RUN script-----------
sufx = str(dt.datetime.now().strftime('%Y%m%d_%H%M'))

output_fc_name = f'InrixPrdSpd_{speed_data_year}on{TMC_yr}TMC{sufx}'


sedf_links = pd.DataFrame.spatial.from_featureclass(input_link_fc)
sedf_links = sedf_links[link_fc_cols]
sedf_links = sedf_links.merge(df_master, how='left', left_on=links_jnky, right_on=join_key)
sedf_links.spatial.to_featureclass(os.path.join(output_fgdb, output_fc_name))

'Q:\\SACSIM23\\Network\\SM23GIS\\SM23Testing.gdb\\InrixPrdSpd_2016on2020TMC20220517_0705'